<a href="https://colab.research.google.com/github/jonik2909/computer-vision/blob/main/menu_detector_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("menu detector")

menu detector


In [9]:
# import libraries

from google.colab import drive
import torchvision.transforms as transforms
from PIL import Image, UnidentifiedImageError
from torch.utils.data import Dataset
import os
import numpy as np

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# Define Dataset Path

DATASET_PATH = '/content/drive/MyDrive/food101_dataset'
print("Dataset Path:", DATASET_PATH)

CUSTOM_CLASS_MAPPING = {
    'hamburger': 'hamburger',
    'hot_dog': 'hot_dog',
    'chocolate_cake': 'dessert', # label grouping | class consolidation
    'cheesecake': 'dessert', # label grouping | class consolidation
    'kebab': 'kebab',
    'pilaf': 'pilaf'
}

CLASSES = ['hamburger', 'hot_dog', 'dessert', 'kebab', 'pilaf']

CLASS_TO_IDX = {c: i for i, c in enumerate(CLASSES)}
NUM_CLASSES = len(CLASSES)

print("Classes:", CLASSES)
print("Class to Index Mapping:", CLASS_TO_IDX)
print("Number of Classes:", NUM_CLASSES)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



Dataset Path: /content/drive/MyDrive/food101_dataset
Classes: ['hamburger', 'hot_dog', 'dessert', 'kebab', 'pilaf']
Class to Index Mapping: {'hamburger': 0, 'hot_dog': 1, 'dessert': 2, 'kebab': 3, 'pilaf': 4}
Number of Classes: 5


In [10]:
# ---------------------------
# Custom Dataset Class
# ---------------------------

class FoodDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        print('images_length', len(self.images))
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        print('image_path', img_path)
        label = self.labels[idx]
        print('label', label)
        try:
            image = Image.open(img_path).convert('RGB') # 3 Channel
        except (UnidentifiedImageError, OSError):
            print(f"Skipping broken image: {img_path}")
            return self.__getitem__((idx + 1) % len(self.images)) #

        if self.transform:
            image = self.transform(image)
        return image, label

In [13]:
# ---------------------------
# Gather and Split Data
# ---------------------------
all_images = []
for original_class, mapped_class in CUSTOM_CLASS_MAPPING.items():
    class_path = os.path.join(DATASET_PATH, original_class) # /content/drive/MyDrive/food101_dataset/hamburger
    print('class_path:', class_path)
    if not os.path.exists(class_path):
        print(f"Warning: {class_path} not found")
        continue
    for img in os.listdir(class_path):
        if img.endswith(('.jpg', '.jpeg', '.png')): # .txt, .docx
            full_path = os.path.join(class_path, img) # /content/drive/MyDrive/food101_dataset/hamburger/100057.jpg
            all_images.append((full_path, CLASS_TO_IDX[mapped_class]))

np.random.shuffle(all_images)
split = int(0.8 * len(all_images))
train_data = all_images[:split] # 1000 | 800 train_data | 200 val_data
val_data = all_images[split:]

train_images, train_labels = zip(*train_data)
val_images, val_labels = zip(*val_data)

print('all_images:', all_images)

dataset = FoodDataset(train_images, train_labels)
print(len(dataset))
img, lbl = dataset[0]

class_path: /content/drive/MyDrive/food101_dataset/hamburger
class_path: /content/drive/MyDrive/food101_dataset/hot_dog
class_path: /content/drive/MyDrive/food101_dataset/chocolate_cake
class_path: /content/drive/MyDrive/food101_dataset/cheesecake
class_path: /content/drive/MyDrive/food101_dataset/kebab
class_path: /content/drive/MyDrive/food101_dataset/pilaf
all_images: [('/content/drive/MyDrive/food101_dataset/hamburger/607907.jpg', 0), ('/content/drive/MyDrive/food101_dataset/cheesecake/1102384.jpg', 2), ('/content/drive/MyDrive/food101_dataset/hamburger/986085.jpg', 0), ('/content/drive/MyDrive/food101_dataset/hot_dog/1233688.jpg', 1), ('/content/drive/MyDrive/food101_dataset/chocolate_cake/3698195.jpg', 2), ('/content/drive/MyDrive/food101_dataset/hot_dog/317564.jpg', 1), ('/content/drive/MyDrive/food101_dataset/chocolate_cake/663983.jpg', 2), ('/content/drive/MyDrive/food101_dataset/hamburger/587018.jpg', 0), ('/content/drive/MyDrive/food101_dataset/pilaf/Image_7.jpg', 4), ('/con